In [1]:
import pandas as pd 

In [2]:
ds = pd.read_csv("antwerp_ds_modified.csv", index_col=0) 

In [3]:
ds

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 64,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,-200,2017-12-28 16:14:07.247000+00:00,12,1.02,51.216736,4.412130
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-11-28 12:01:02.380000+00:00,7,0.57,51.232037,4.405185
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-14 09:05:22.832000+00:00,9,1.00,51.202763,4.390477
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,-200,-200,-200,-200,-200,2018-01-17 09:11:49.222000+00:00,12,0.62,51.194584,4.417887
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-10 14:47:01.379000+00:00,12,1.00,51.207203,4.403470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,-200,-103,-106,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-04 10:57:15.856000+00:00,12,0.61,51.201839,4.414310
123524,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-03 23:17:34.799000+00:00,8,1.10,51.233677,4.416892
123525,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-15 10:13:56.383000+00:00,12,0.82,51.215370,4.412519
123526,-200,-97,-200,-200,-200,-200,-200,-200,-200,-92,...,-200,-200,-200,-200,-200,2018-01-12 23:33:09.229000+00:00,12,0.61,51.194252,4.418354


In [4]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [5]:
# Convert the RX Time values to datetime format ISO8601 and store in a separate column upto minutes precision
# Expected format "2017-12-28T16:14"
ds["Weather param sense time"] = pd.to_datetime(ds["RX Time"], format='ISO8601').dt.strftime('%Y-%m-%dT%H:%M')

In [6]:
f"{ds["Weather param sense time"][0]}"

'2017-12-28T16:14'

# Weather data collector

def get_weather_info_mod(ds_row):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    # url = "https://archive-api.open-meteo.com/v1/archive"  # this url is for free api calls 
    # For commercial usage the url below must be used
    url = "https://customer-archive-api.open-meteo.com/v1/archive" 

    params = {
        "latitude": ds_row["Latitude"],
        "longitude": ds_row["Longitude"],
        "elevation": 5.0,
        "start_hour": f"{ds_row["Weather param sense time"]}",
        "end_hour": f"{ds_row["Weather param sense time"]}",
        "hourly": ["temperature_2m", 
                "relative_humidity_2m",
                "dew_point_2m", 
                "apparent_temperature", 
                "precipitation", 
                "rain", 
                "snowfall", 
                "snow_depth", 
                "weather_code", 
                "pressure_msl", 
                "surface_pressure", 
                "cloud_cover", 
                "cloud_cover_low", 
                "cloud_cover_mid", 
                "cloud_cover_high", 
                "et0_fao_evapotranspiration", 
                "vapour_pressure_deficit", 
                "wind_speed_10m", 
                "wind_speed_100m", 
                "wind_direction_10m", 
                "wind_direction_100m", 
                "wind_gusts_10m", 
                "shortwave_radiation", 
                "direct_radiation",
                "diffuse_radiation", 
                "direct_normal_irradiance", 
                "global_tilted_irradiance", 
                "terrestrial_radiation", 
                "shortwave_radiation_instant",
                "direct_radiation_instant", 
                "diffuse_radiation_instant", 
                "direct_normal_irradiance_instant", 
                "global_tilted_irradiance_instant", 
                "terrestrial_radiation_instant"],
        "apikey": "Y8Z47uEDxQ7akhwt"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(22).ValuesAsNumpy()
    hourly_direct_radiation = hourly.Variables(23).ValuesAsNumpy()
    hourly_diffuse_radiation = hourly.Variables(24).ValuesAsNumpy()
    hourly_direct_normal_irradiance = hourly.Variables(25).ValuesAsNumpy()
    hourly_global_tilted_irradiance = hourly.Variables(26).ValuesAsNumpy()
    hourly_terrestrial_radiation = hourly.Variables(27).ValuesAsNumpy()
    hourly_shortwave_radiation_instant = hourly.Variables(28).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(29).ValuesAsNumpy()
    hourly_diffuse_radiation_instant = hourly.Variables(30).ValuesAsNumpy()
    hourly_direct_normal_irradiance_instant = hourly.Variables(31).ValuesAsNumpy()
    hourly_global_tilted_irradiance_instant = hourly.Variables(32).ValuesAsNumpy()
    hourly_terrestrial_radiation_instant = hourly.Variables(33).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
    hourly_data["direct_radiation"] = hourly_direct_radiation
    hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
    hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
    hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
    hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
    hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
    hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
    hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
    hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
    hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant

    hourly_dataframe = pd.DataFrame(data = hourly_data, index=[ds_row.name])  # Here we set index to the value of the original row index
    # this will help us concatinating the weather data to the main dataset
    # the index are only identifier of each sample and their corresponding weather data 
    return hourly_dataframe

In [7]:
ds

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude,Weather param sense time
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,2017-12-28 16:14:07.247000+00:00,12,1.02,51.216736,4.412130,2017-12-28T16:14
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-11-28 12:01:02.380000+00:00,7,0.57,51.232037,4.405185,2017-11-28T12:01
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-12-14 09:05:22.832000+00:00,9,1.00,51.202763,4.390477,2017-12-14T09:05
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,-200,-200,-200,-200,2018-01-17 09:11:49.222000+00:00,12,0.62,51.194584,4.417887,2018-01-17T09:11
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-10 14:47:01.379000+00:00,12,1.00,51.207203,4.403470,2018-01-10T14:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,-200,-103,-106,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-04 10:57:15.856000+00:00,12,0.61,51.201839,4.414310,2018-01-04T10:57
123524,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-03 23:17:34.799000+00:00,8,1.10,51.233677,4.416892,2018-01-03T23:17
123525,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-12-15 10:13:56.383000+00:00,12,0.82,51.215370,4.412519,2017-12-15T10:13
123526,-200,-97,-200,-200,-200,-200,-200,-200,-200,-92,...,-200,-200,-200,-200,2018-01-12 23:33:09.229000+00:00,12,0.61,51.194252,4.418354,2018-01-12T23:33


# Sample of n are tested for getting weather data
sample_ds = ds.sample(n=4000)
rdf = pd.concat([get_weather_info_mod(sample_ds.iloc[i, :]) for i in  range(len(sample_ds))], axis=0)

# Weather Data collection 
## Change cell to code 

import time
import datetime


# Sample of n are tested for getting weather data
# sample_ds = ds.iloc[:4000]
sample_ds = ds.iloc[10007:]

rdf = pd.DataFrame()

i = 0
while i < len(sample_ds):

    try:
        rdf = pd.concat([rdf, get_weather_info_mod(sample_ds.iloc[i, :])], axis=0)
        print(f'saving data{i}')
        rdf.to_csv('weather_data_licenced_2.csv')
        i += 1  
    except Exception as e:
        if 'Daily API request limit exceeded. Please try again tomorrow' in str(e):
            print(f'waiting untill {datetime.datetime.now() + datetime.timedelta(days=1)}')
            time.sleep(datetime.timedelta(days=1).total_seconds())

        elif 'Hourly API request limit exceeded. Please try again in the next hour' in str(e):
            print(f'waiting untill {datetime.datetime.now() + datetime.timedelta(hours=1)}')
            time.sleep(datetime.timedelta(hours=1).total_seconds())

        elif 'Minutely API request limit exceeded. Please try again in one minute' in str(e):
            print(f'waiting untill {datetime.datetime.now() + datetime.timedelta(minutes=1)}')
            time.sleep(datetime.timedelta(minutes=1).total_seconds())

        else:
            raise e

In [9]:
# Reading the weather information from local csv files 
weather_ds1 = pd.read_csv('weather_data_licenced_1.csv', index_col=0)

weather_ds1

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,2017-12-28 16:14:00+00:00,3.2455,74.766975,-0.8045,-1.471424,0.0,0.0,0.00,0.00,1.0,...,5.0,51.513760,7.999999,21.081083,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,2017-11-28 12:01:00+00:00,6.6955,85.262260,4.3955,3.050040,0.0,0.0,0.00,0.00,1.0,...,79.0,356.052460,184.999980,418.422520,182.836150,104.760170,78.075970,356.052500,182.836140,413.52840
2,2017-12-14 09:05:00+00:00,4.0455,79.654655,0.8455,-2.085968,0.0,0.0,0.00,0.01,2.0,...,29.0,54.315105,35.000004,133.442540,54.010590,9.258959,44.751633,63.470764,46.219566,205.92316
3,2018-01-17 09:11:00+00:00,3.2455,72.081696,-1.3045,-3.012658,0.2,0.0,0.14,0.00,71.0,...,24.0,170.033160,44.999992,164.430280,66.079580,30.837140,35.242443,180.502980,62.246716,241.45522
4,2018-01-10 14:47:00+00:00,7.4455,94.019670,6.5455,4.709943,0.1,0.1,0.00,0.00,51.0,...,35.0,11.516722,37.000000,245.607710,26.762775,1.446637,25.316140,11.516721,26.762775,177.65254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10002,2018-02-03 20:48:00+00:00,1.3455,96.470320,0.8455,-1.260212,0.0,0.0,0.00,0.00,2.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
10003,2018-01-17 10:46:00+00:00,3.9955,71.701760,-0.6545,-2.429144,0.1,0.1,0.00,0.00,51.0,...,56.0,81.421170,77.000000,364.523040,85.429600,23.298983,62.130623,81.421165,85.429600,404.42935
10004,2017-12-19 13:11:00+00:00,5.5955,88.815704,3.8955,2.084299,0.0,0.0,0.00,0.00,3.0,...,52.0,127.490200,83.000010,343.559900,75.146410,28.066730,47.079678,127.490200,75.146416,311.05170
10005,2018-02-01 10:39:00+00:00,3.5955,82.214510,0.8455,-1.476644,0.0,0.0,0.00,0.00,1.0,...,84.0,226.442670,152.999980,429.334720,169.138600,76.278190,92.860405,226.442670,169.138580,474.62130


In [10]:
# Reading the weather information from local csv files 
weather_ds2 = pd.read_csv('weather_data_licenced_2.csv', index_col=0)

weather_ds2

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
10007,2018-01-08 08:09:00+00:00,1.945500,80.806500,-1.0045,-3.317702,0.0,0.0,0.00,0.00,1.0,...,0.0,0.000000,0.00000,16.698757,0.000000,0.000000,0.000000,0.000000,0.000000,53.156666
10008,2018-01-05 11:10:00+00:00,8.695499,80.883630,5.5955,4.968346,0.1,0.1,0.00,0.00,51.0,...,96.0,31.716606,104.00000,356.808380,111.781250,8.598557,103.182690,31.716606,111.781250,383.504670
10009,2018-02-02 07:24:00+00:00,3.895500,91.552470,2.6455,-1.245896,1.6,1.3,0.21,0.00,73.0,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10010,2018-01-30 09:58:00+00:00,3.045500,95.494690,2.3955,-0.018787,0.0,0.0,0.00,0.00,0.0,...,37.0,241.800540,94.99999,338.150240,112.467360,68.664280,43.803078,241.800540,112.467354,400.324920
10011,2018-01-12 16:34:00+00:00,5.895500,92.646904,4.7955,3.845880,0.0,0.0,0.00,0.00,3.0,...,8.0,0.000000,8.00000,19.157300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43666,2018-01-23 08:32:00+00:00,6.345500,91.710980,5.0955,2.636001,0.0,0.0,0.00,0.00,3.0,...,1.0,0.000000,1.00000,74.171510,2.209313,0.000000,2.209313,0.000000,1.226738,163.868060
43667,2017-12-14 08:44:00+00:00,3.945500,75.436485,-0.0045,-2.280131,0.0,0.0,0.00,0.01,3.0,...,1.0,0.000000,1.00000,78.164290,2.014187,0.000000,2.014187,0.000000,1.207201,157.437530
43668,2017-12-06 08:37:00+00:00,5.845500,89.775510,4.2955,2.308942,0.0,0.0,0.00,0.00,1.0,...,2.0,10.525630,3.00000,83.759840,5.886318,1.962106,3.924212,16.820736,3.683383,164.345690
43669,2017-12-07 09:00:00+00:00,3.545500,86.138596,1.4455,-1.832110,0.1,0.1,0.00,0.00,51.0,...,23.0,17.562763,25.00000,141.449890,37.956590,3.036527,34.920063,19.926365,33.454300,214.758210


In [11]:
# Reading the weather information from local csv files 
weather_ds3 = pd.read_csv('weather_data_licenced_3.csv', index_col=0)

weather_ds3

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
43671,2018-01-04 15:57:00+00:00,11.8955,86.974174,9.7955,7.298428,0.0,0.0,0.0,0.0,3.0,...,29.0,0.00000,29.000000,56.105420,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
43672,2018-01-10 07:22:00+00:00,7.2455,91.768110,5.9955,3.932602,1.0,1.0,0.0,0.0,55.0,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
43673,2018-01-12 11:28:00+00:00,5.6955,97.602530,5.3455,4.245604,0.0,0.0,0.0,0.0,3.0,...,33.0,0.00000,33.000000,393.875000,34.671326,0.000000,34.671326,0.00000,34.671326,413.82324
43674,2018-01-17 16:29:00+00:00,4.3150,73.611210,0.0150,-1.297543,0.2,0.2,0.0,0.0,51.0,...,15.0,227.89267,31.000006,38.561638,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
43675,2017-11-28 13:09:00+00:00,5.9955,84.290520,3.5455,2.089982,1.0,1.0,0.0,0.0,55.0,...,88.0,305.34660,170.999980,382.045650,154.905270,75.187940,79.717330,305.34660,154.905260,346.08707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,2018-01-04 10:57:00+00:00,7.6955,94.354390,6.8455,4.421107,0.7,0.7,0.0,0.0,53.0,...,12.0,0.00000,12.000000,340.036320,13.140089,0.000000,13.140089,0.00000,13.140089,372.34230
123524,2018-01-03 23:17:00+00:00,7.2955,65.928030,1.3455,1.338683,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
123525,2017-12-15 10:13:00+00:00,3.5955,92.514110,2.4955,0.219360,0.0,0.0,0.0,0.0,3.0,...,68.0,143.33075,96.000000,275.819850,112.295120,32.752743,79.542370,143.33075,112.295120,322.63773
123526,2018-01-12 23:33:00+00:00,4.9455,96.908590,4.4955,2.345604,0.0,0.0,0.0,0.0,3.0,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000


In [12]:
weather_info = pd.concat([weather_ds1, weather_ds2, weather_ds3], axis=0)

weather_info

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,2017-12-28 16:14:00+00:00,3.2455,74.766975,-0.8045,-1.471424,0.0,0.0,0.00,0.00,1.0,...,5.0,51.513760,7.999999,21.081083,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,2017-11-28 12:01:00+00:00,6.6955,85.262260,4.3955,3.050040,0.0,0.0,0.00,0.00,1.0,...,79.0,356.052460,184.999980,418.422520,182.836150,104.760170,78.075970,356.052500,182.836140,413.52840
2,2017-12-14 09:05:00+00:00,4.0455,79.654655,0.8455,-2.085968,0.0,0.0,0.00,0.01,2.0,...,29.0,54.315105,35.000004,133.442540,54.010590,9.258959,44.751633,63.470764,46.219566,205.92316
3,2018-01-17 09:11:00+00:00,3.2455,72.081696,-1.3045,-3.012658,0.2,0.0,0.14,0.00,71.0,...,24.0,170.033160,44.999992,164.430280,66.079580,30.837140,35.242443,180.502980,62.246716,241.45522
4,2018-01-10 14:47:00+00:00,7.4455,94.019670,6.5455,4.709943,0.1,0.1,0.00,0.00,51.0,...,35.0,11.516722,37.000000,245.607710,26.762775,1.446637,25.316140,11.516721,26.762775,177.65254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,2018-01-04 10:57:00+00:00,7.6955,94.354390,6.8455,4.421107,0.7,0.7,0.00,0.00,53.0,...,12.0,0.000000,12.000000,340.036320,13.140089,0.000000,13.140089,0.000000,13.140089,372.34230
123524,2018-01-03 23:17:00+00:00,7.2955,65.928030,1.3455,1.338683,0.0,0.0,0.00,0.00,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
123525,2017-12-15 10:13:00+00:00,3.5955,92.514110,2.4955,0.219360,0.0,0.0,0.00,0.00,3.0,...,68.0,143.330750,96.000000,275.819850,112.295120,32.752743,79.542370,143.330750,112.295120,322.63773
123526,2018-01-12 23:33:00+00:00,4.9455,96.908590,4.4955,2.345604,0.0,0.0,0.00,0.00,3.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [13]:
ds_weather = pd.concat([ds, weather_info], axis=1, join='outer')

In [14]:
ds_weather

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,5.0,51.513760,7.999999,21.081083,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,79.0,356.052460,184.999980,418.422520,182.836150,104.760170,78.075970,356.052500,182.836140,413.52840
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,29.0,54.315105,35.000004,133.442540,54.010590,9.258959,44.751633,63.470764,46.219566,205.92316
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,24.0,170.033160,44.999992,164.430280,66.079580,30.837140,35.242443,180.502980,62.246716,241.45522
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,35.0,11.516722,37.000000,245.607710,26.762775,1.446637,25.316140,11.516721,26.762775,177.65254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,-200,-103,-106,-200,-200,-200,-200,-200,-200,-200,...,12.0,0.000000,12.000000,340.036320,13.140089,0.000000,13.140089,0.000000,13.140089,372.34230
123524,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
123525,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,68.0,143.330750,96.000000,275.819850,112.295120,32.752743,79.542370,143.330750,112.295120,322.63773
123526,-200,-97,-200,-200,-200,-200,-200,-200,-200,-92,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [15]:
ds_weather.to_csv('antwerp_ds_weather-data.csv')